Manipulando dados para serem processador no Machine Learning

Imports

In [354]:
import unicodedata
import re
import os
import numpy as np
import pandas as pd

Ler Excel

In [355]:
df = pd.read_excel('laudos_piloto.xlsx')

Limpando dados nulos que não foram coletadas corretamente

In [356]:
df.isnull().sum()

ID                       0
Nome do Paciente         0
Nome do Tutor          190
Idade                  190
Espécie                  0
Raça                    10
Regiao                   0
Posicionamentos          0
Laudo                    0
Exame                    6
Numero de Projeções     65
Data do Exame            6
Imagens                  6
dtype: int64

In [357]:
df = df.dropna(subset=['Laudo', 'Imagens','Exame', 'Raça','Idade'])

In [358]:
df.shape

(2418, 13)

In [359]:
df.isnull().sum()

ID                      0
Nome do Paciente        0
Nome do Tutor           0
Idade                   0
Espécie                 0
Raça                    0
Regiao                  0
Posicionamentos         0
Laudo                   0
Exame                   0
Numero de Projeções    56
Data do Exame           0
Imagens                 0
dtype: int64

Alterando idade para meses

In [360]:
def idade_para_meses(idade_str):
    if pd.isnull(idade_str):
        return None
    idade_str = idade_str.lower()
    anos = re.search(r'(\d+)\s*ano', idade_str)
    meses = re.search(r'(\d+)\s*mes', idade_str)

    total_meses = 0
    if anos:
        total_meses += int(anos.group(1)) * 12
    if meses:
        total_meses += int(meses.group(1))
    
    return total_meses if total_meses > 0 else None

In [361]:
df['Idade (meses)'] = df['Idade'].apply(idade_para_meses)

In [362]:
df.drop(columns=['Idade'], inplace=True)


In [363]:
df.head(1)

,ID,Nome do Paciente,Nome do Tutor,Espécie,Raça,Regiao,Posicionamentos,Laudo,Exame,Numero de Projeções,Data do Exame,Imagens,Idade (meses)
0,1,BALTAZAR,ANDRESSA,CANINO,CAVALIER KING CHARLES,Coluna vertebral (segmento cervicotorácica e t...,Laterolateral (decúbito lateral direito) e ven...,Diminuição do espaço intervertebral entre T11-...,COLUNA,5P,29/04/2025,001_BALTAZAR ANDRESSA 12 anos 17225 29_04_2025...,144


Contando quantidade de imagens e colocando dentro de uma coluna


In [364]:
df['lista de imagens'] = df['Imagens'].apply(
    lambda x: [img.strip() for img in str(x).split(',')] if pd.notnull(x) else []
)

In [365]:
df['quantidade de imagens'] = df['lista de imagens'].apply(len)

In [366]:
df.shape

(2418, 15)

In [367]:
df.isnull().sum()

ID                        0
Nome do Paciente          0
Nome do Tutor             0
Espécie                   0
Raça                      0
Regiao                    0
Posicionamentos           0
Laudo                     0
Exame                     0
Numero de Projeções      56
Data do Exame             0
Imagens                   0
Idade (meses)             0
lista de imagens          0
quantidade de imagens     0
dtype: int64

In [368]:
df = df.dropna(subset=['Idade (meses)', 'Exame'])

In [369]:
df.isnull().sum()

ID                        0
Nome do Paciente          0
Nome do Tutor             0
Espécie                   0
Raça                      0
Regiao                    0
Posicionamentos           0
Laudo                     0
Exame                     0
Numero de Projeções      56
Data do Exame             0
Imagens                   0
Idade (meses)             0
lista de imagens          0
quantidade de imagens     0
dtype: int64

Separando exames com alteração em laudo e sem alteração no laudo

In [370]:
def verificar_alteracao(laudo):
    palavras_alteracao = ['fratura', 'luxação', 'subluxação', 'desvio', 'deslocamento',
        'lesão', 'lesões', 'infiltração', 'inflamação', 'calcificação',
        'osteofitose', 'osteólise', 'osteopenia', 'osteoporose', 'edema',
        'compressão', 'colapso', 'protusão', 'extrusão', 'degeneração',
        'esclerose', 'hiperplasia', 'hipoplasia', 'hipodensidade', 'hiperdensidade',
        'afundamento', 'espaçamento reduzido', 'desalinhamento', 'alinhamento anormal',
        'massas', 'tumoração', 'nódulo', 'nodularidade', 'irregularidade',
        'espessamento', 'redução de espaço articular', 'calcificações',
        'contusão', 'hemorragia', 'distensão', 'distúrbio', 'erosão', 'artrite',
        'artrose', 'necrose', 'proliferação óssea', 'aplasia', 'estenose',
        'encurtamento', 'malformação', 'incongruência articular',
        'litíase', 'urolitíase', 'colelitíase', 'pneumotórax', 'hidrotórax',
        'pneumonia', 'atelectasia', 'derrame pleural', 'bronquite', 'broncopneumonia',
        'densificação', 'opacificação', 'hipotransparência', 'hipertransparência',
        'linfadenomegalia', 'hepatomegalia', 'esplenomegalia', 'megacólon', 'gás livre',
        'retenção fecal', 'dilatação', 'intussuscepção', 'volvo', 'torsão', 'ruptura',
        'derrame', 'alça distendida', 'assimetrias', 'calcificação distrófica',
        'hipervascularização', 'condromalácia', 'sinais de trauma', 'sinais de instabilidade',
        'espessamento pleural', 'parênquima irregular', 'massa pulmonar', 'bronquiectasia',
        'cisto', 'condensação', 'hipoecogenicidade', 'hiperecogenicidade', 'ectasia',
        'metástase', 'infiltrado', 'infiltrado intersticial', 'obliteração', 'reabsorção óssea',
        'descontinuidade cortical', 'linhas de fratura', 'sinais de infecção', 'proeminência óssea'
    ]
    
    # Palavras que indicam normalidade
    palavras_normais = ['normal', 'normais', 'sem alterações', 'sem alteracoes', 
                       'dentro da normalidade', 'dentro dos padrões de normalidade',
                       'sem anormalidades', 'sem anormalidades detectadas',
                       'adequado', 'adequados', 'preservado', 'preservados',
                       'sem sinais de', 'ausência de', 'ausencia de']
    
    laudo = str(laudo).lower()
    laudo_limpo = re.sub(r'[^\w\s]', ' ', laudo)  # Remove pontuação
    
    # Verificar se há indicação explícita de normalidade
    for palavra_normal in palavras_normais:
        if palavra_normal in laudo:
            # Se encontrou palavra normal, verificar se não há contradição
            palavras_encontradas = []
            for palavra in palavras_alteracao:
                # Padrão de negação
                padrao_negado = r'(sem|não|nao|ausência de|ausencia de|negativo para|descartado|descartada)\s+(\w+\s+){0,5}' + re.escape(palavra)
                padrao_afirmativo = r'\b' + re.escape(palavra) + r'\b'
                
                if re.search(padrao_negado, laudo):
                    continue  # Palavra negada, não conta
                if re.search(padrao_afirmativo, laudo):
                    palavras_encontradas.append(palavra)
            
            # Se encontrou normalidade mas também alterações, priorizar alterações
            if palavras_encontradas:
                return 1
            else:
                return 0
    
    # Se não há indicação explícita de normalidade, procurar por alterações
    for palavra in palavras_alteracao:
        # Padrão de negação mais robusto
        padrao_negado = r'(sem|não|nao|ausência de|ausencia de|negativo para|descartado|descartada)\s+(\w+\s+){0,5}' + re.escape(palavra)
        padrao_afirmativo = r'\b' + re.escape(palavra) + r'\b'
        
        if re.search(padrao_negado, laudo):
            continue  # Palavra negada, não conta
        if re.search(padrao_afirmativo, laudo):
            return 1
    
    return 0

In [371]:
df['Laudo Alterado'] = df['Laudo'].apply(verificar_alteracao)

In [372]:
df['Laudo Alterado'].value_counts()

Laudo Alterado
1    1583
0     835
Name: count, dtype: int64

Padronizando nomes

In [373]:
def padronizar_especie(valor):
    valor = str(valor).strip().lower()

    if 'fel' in valor:
        return 'FELINO'
    elif 'can' in valor:
        return 'CANINO'
    else:
        return valor

In [374]:
df['Espécie'] = df['Espécie'].apply(padronizar_especie)

In [375]:
df.shape

(2418, 16)

In [376]:
def remove_duplicatas(texto):
    if isinstance(texto, str):
        palavras = texto.split()
        palavras_unicas = []
        for p in palavras:
            if p not in palavras_unicas:
                palavras_unicas.append(p)
        return ' '.join(palavras_unicas)
    return texto

In [377]:
df['Exame'] = df['Exame'].apply(remove_duplicatas)

In [378]:
df.shape

(2418, 16)

Exame separado por '+'

In [379]:
df = df.rename(columns={'Exame': 'Todos Exames'})

In [380]:
def separar_exames(valor):
    if pd.isna(valor) or str(valor).strip() == '':
        return []
    return [item.strip() for item in str(valor).split('+')]

Separar Exames

In [381]:
df['Exames Separados'] = df['Todos Exames'].apply(separar_exames)

Determinar o número máximo de exames para definir quantas colunas criar

In [382]:
max_exames = df['Exames Separados'].apply(len).max()

Criar novas colunas com base no número de exames

In [383]:
nomes_colunas = ['Primeira Região', 'Segunda Região', 'Terceira Região', 
                'Quarta Região', 'Quinta Região', 'Sexta Região']

for i in range(max_exames):
    if i < len(nomes_colunas):
        nome_coluna = nomes_colunas[i]
    else:
        nome_coluna = f'Região {i + 1}'
    df[nome_coluna] = df['Exames Separados'].apply(lambda x: x[i] if i < len(x) else None)

In [384]:
df.head(10)

,ID,Nome do Paciente,Nome do Tutor,Espécie,Raça,Regiao,Posicionamentos,Laudo,Todos Exames,Numero de Projeções,...,Imagens,Idade (meses),lista de imagens,quantidade de imagens,Laudo Alterado,Exames Separados,Primeira Região,Segunda Região,Terceira Região,Quarta Região
0,1,BALTAZAR,ANDRESSA,CANINO,CAVALIER KING CHARLES,Coluna vertebral (segmento cervicotorácica e t...,Laterolateral (decúbito lateral direito) e ven...,Diminuição do espaço intervertebral entre T11-...,COLUNA,5P,...,001_BALTAZAR ANDRESSA 12 anos 17225 29_04_2025...,144,[001_BALTAZAR ANDRESSA 12 anos 17225 29_04_202...,5,0,[COLUNA],COLUNA,None,None,None
1,2,ESMERALDA,WILLIAN,FELINO,SRD,Tórax,"Laterolateral direita, esquerda e ventrodorsal",Sem evidências de nódulos no parênquima pulmon...,TORAX,3P,...,002_ESMERALDA WILLIAN 14 anos 17185 28_04_2025...,168,[002_ESMERALDA WILLIAN 14 anos 17185 28_04_202...,3,1,[TORAX],TORAX,None,None,None
2,3,LUNA,RENATA,CANINO,CANE CORSO,membro torácico esquerdo (membro direito para ...,mediolateral e dorsopalmar.,-proliferação óssea sólida e regular adjacente...,MTE + CONTRA,4P,...,003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_i...,120,[003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_...,4,1,"[MTE, CONTRA]",MTE,CONTRA,None,None
3,4,MALTA,GUILHERME,CANINO,DUSCHUND,Tórax e coluna vertebral (segmento coccígeo),"Laterolateral direita, esquerda e ventrodorsal",- Tórax: Opacificação pulmonar generalizada de...,TORAX + CAUDA,6P,...,004_MALTA GUILHERME 7 anos 17230 29_04_2025_pg...,84,[004_MALTA GUILHERME 7 anos 17230 29_04_2025_p...,6,1,"[TORAX, CAUDA]",TORAX,CAUDA,None,None
4,5,AIRAM,WALTON,FELINO,SRD,Abdômen,Laterolateral (decúbito lateral direito e esqu...,Maior homogeneidade em cavidade abdominal. Vis...,ABDOME,3P,...,005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2_...,2,[005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2...,3,0,[ABDOME],ABDOME,None,None,None
5,6,JUJUBA,MARCELO,FELINO,SIAMES,REGIÃO: Tórax.,PROJEÇÕES: Laterolateral (decúbito direito e e...,Achados Radiográficos Campos pulmonares aprese...,TORAX,3P,...,006_JUJUBA MARCELO 11 anos 17064 26_04_2025_pg...,132,[006_JUJUBA MARCELO 11 anos 17064 26_04_2025_p...,3,1,[TORAX],TORAX,None,None,None
6,7,LAURA,ONELI,CANINO,PINSCHER,Cervical (partes moles),Laterolateral e tangencial,Segmento vertebral cervical sem alterações rad...,TRAQUEIA,3P,...,007_LAURA ONELI 6 anos 16026 01_04_2025_pg2_im...,72,[007_LAURA ONELI 6 anos 16026 01_04_2025_pg2_i...,3,1,[TRAQUEIA],TRAQUEIA,None,None,None
7,8,VIDA,YURI,FELINO,TRICOLOR,Abdômen,Laterolateral (decúbito lateral direito e esqu...,Estruturas fetais parcialmente mineralizadas e...,ABDOMEN,3P,...,008_VIDA YURI 6 meses 17279 30_04_2025_pg2_img...,6,[008_VIDA YURI 6 meses 17279 30_04_2025_pg2_im...,3,0,[ABDOMEN],ABDOMEN,None,None,None
8,9,SIMBA,MORGANA,CANINO,ROTWEILLER,membro torácico esquerdo (membro direito para ...,mediolateral e dorsopalmar.,"-lesão óssea agressiva mista, predominentement...",MTE + CONTRA,4P,...,009_SIMBA MORGANA 9 anos 17223 30_04_2025_pg2_...,108,[009_SIMBA MORGANA 9 anos 17223 30_04_2025_pg2...,4,1,"[MTE, CONTRA]",MTE,CONTRA,None,None
9,10,CLEO,PAULA,CANINO,LHASA,Tórax,Laterolateral (decúbito lateral direito e esqu...,Discreta incisura interlobar sobrepondo a silh...,TORAX,4P,...,010_CLEO PAULA 9 anos 17304 30_04_2025_pg2_img...,108,[010_CLEO PAULA 9 anos 17304 30_04_2025_pg2_im...,4,1,[TORAX],TORAX,None,None,None


In [385]:
df.head(10)

,ID,Nome do Paciente,Nome do Tutor,Espécie,Raça,Regiao,Posicionamentos,Laudo,Todos Exames,Numero de Projeções,...,Imagens,Idade (meses),lista de imagens,quantidade de imagens,Laudo Alterado,Exames Separados,Primeira Região,Segunda Região,Terceira Região,Quarta Região
0,1,BALTAZAR,ANDRESSA,CANINO,CAVALIER KING CHARLES,Coluna vertebral (segmento cervicotorácica e t...,Laterolateral (decúbito lateral direito) e ven...,Diminuição do espaço intervertebral entre T11-...,COLUNA,5P,...,001_BALTAZAR ANDRESSA 12 anos 17225 29_04_2025...,144,[001_BALTAZAR ANDRESSA 12 anos 17225 29_04_202...,5,0,[COLUNA],COLUNA,None,None,None
1,2,ESMERALDA,WILLIAN,FELINO,SRD,Tórax,"Laterolateral direita, esquerda e ventrodorsal",Sem evidências de nódulos no parênquima pulmon...,TORAX,3P,...,002_ESMERALDA WILLIAN 14 anos 17185 28_04_2025...,168,[002_ESMERALDA WILLIAN 14 anos 17185 28_04_202...,3,1,[TORAX],TORAX,None,None,None
2,3,LUNA,RENATA,CANINO,CANE CORSO,membro torácico esquerdo (membro direito para ...,mediolateral e dorsopalmar.,-proliferação óssea sólida e regular adjacente...,MTE + CONTRA,4P,...,003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_i...,120,[003_LUNA RENATA 10 anos 17231 29_04_2025_pg2_...,4,1,"[MTE, CONTRA]",MTE,CONTRA,None,None
3,4,MALTA,GUILHERME,CANINO,DUSCHUND,Tórax e coluna vertebral (segmento coccígeo),"Laterolateral direita, esquerda e ventrodorsal",- Tórax: Opacificação pulmonar generalizada de...,TORAX + CAUDA,6P,...,004_MALTA GUILHERME 7 anos 17230 29_04_2025_pg...,84,[004_MALTA GUILHERME 7 anos 17230 29_04_2025_p...,6,1,"[TORAX, CAUDA]",TORAX,CAUDA,None,None
4,5,AIRAM,WALTON,FELINO,SRD,Abdômen,Laterolateral (decúbito lateral direito e esqu...,Maior homogeneidade em cavidade abdominal. Vis...,ABDOME,3P,...,005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2_...,2,[005_AIRAM WALTON 2 meses 17191 28_04_2025_pg2...,3,0,[ABDOME],ABDOME,None,None,None
5,6,JUJUBA,MARCELO,FELINO,SIAMES,REGIÃO: Tórax.,PROJEÇÕES: Laterolateral (decúbito direito e e...,Achados Radiográficos Campos pulmonares aprese...,TORAX,3P,...,006_JUJUBA MARCELO 11 anos 17064 26_04_2025_pg...,132,[006_JUJUBA MARCELO 11 anos 17064 26_04_2025_p...,3,1,[TORAX],TORAX,None,None,None
6,7,LAURA,ONELI,CANINO,PINSCHER,Cervical (partes moles),Laterolateral e tangencial,Segmento vertebral cervical sem alterações rad...,TRAQUEIA,3P,...,007_LAURA ONELI 6 anos 16026 01_04_2025_pg2_im...,72,[007_LAURA ONELI 6 anos 16026 01_04_2025_pg2_i...,3,1,[TRAQUEIA],TRAQUEIA,None,None,None
7,8,VIDA,YURI,FELINO,TRICOLOR,Abdômen,Laterolateral (decúbito lateral direito e esqu...,Estruturas fetais parcialmente mineralizadas e...,ABDOMEN,3P,...,008_VIDA YURI 6 meses 17279 30_04_2025_pg2_img...,6,[008_VIDA YURI 6 meses 17279 30_04_2025_pg2_im...,3,0,[ABDOMEN],ABDOMEN,None,None,None
8,9,SIMBA,MORGANA,CANINO,ROTWEILLER,membro torácico esquerdo (membro direito para ...,mediolateral e dorsopalmar.,"-lesão óssea agressiva mista, predominentement...",MTE + CONTRA,4P,...,009_SIMBA MORGANA 9 anos 17223 30_04_2025_pg2_...,108,[009_SIMBA MORGANA 9 anos 17223 30_04_2025_pg2...,4,1,"[MTE, CONTRA]",MTE,CONTRA,None,None
9,10,CLEO,PAULA,CANINO,LHASA,Tórax,Laterolateral (decúbito lateral direito e esqu...,Discreta incisura interlobar sobrepondo a silh...,TORAX,4P,...,010_CLEO PAULA 9 anos 17304 30_04_2025_pg2_img...,108,[010_CLEO PAULA 9 anos 17304 30_04_2025_pg2_im...,4,1,[TORAX],TORAX,None,None,None


Algumas alterações para padronização

In [386]:
coluna_limpar = ['Primeira Região', 'Segunda Região', 'Terceira Região', 'Quarta Região']

In [387]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['TRAQUEIA', 'COLAPSO', 'COLAPSO DE TRAQUEIA'], 
    'CERVICAL (PARTES MOLES)'
)

In [388]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['MPS'], 
    'MEMBROS PELVICOS'
)

In [389]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['MPE'], 
    'MEMBRO PELVICO ESQUERDO'
)

In [390]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['MPD'], 
    'MEMBRO PELVICO DIREITO'
)

In [391]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['MTD'], 
    'MEMBRO TORACICO DIREITO'
)

In [392]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['MTE'], 
    'MEMBRO TORACICO ESQUERDO'
)

In [393]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['PELVE', 'PELVIS', 'COXOFEMORAL'], 
    'COXAL'
)

In [394]:
df[coluna_limpar] = df[coluna_limpar].replace(
    ['CI'], 
    'CORPO INTEIRO (SILVESTRE)'
)

In [395]:
df['Raça'] = df['Raça'].replace(
    ['CHOW'], 
    'CHOW CHOW'
)

Formalizando Espécies

In [396]:
df['Espécie'] = df['Espécie'].str.upper()

In [397]:
df = df.drop_duplicates(subset=['Nome do Paciente', 'Nome do Tutor', 'Espécie'], keep='last')

In [398]:
df.shape

(1091, 21)

In [399]:
df['Espécie'] = df['Espécie'].replace(
    ['ANINA'], 
    'CANINO'
)

In [400]:
df['Espécie'] = df['Espécie'].replace(
    ['AVES'], 
    'AVE'
)

In [401]:
df['Espécie'] = df['Espécie'].replace(
    ['TARTARUGA'], 
    'QUELONIO'
)

In [402]:
df['Espécie'] = df['Espécie'].replace(
    ['LARGOMORFO', 'COELHO'], 
    'LAGOMORFO'
)

Limpeza da coluna 'Regiao

In [403]:
df['Espécie'] = df['Espécie'].replace(
    ['LAGOMORFO', 'ROEDOR', 'REPTIL'], 
    'SILVESTRE'
)

In [404]:
df['Espécie'].value_counts()

Espécie
CANINO       777
FELINO       156
AVE          100
SILVESTRE     33
QUELONIO      23
QUILONIO       1
CAINO          1
Name: count, dtype: int64

In [405]:
df['Regiao'] = (
    df['Regiao']
    .str.replace(r'\.$', '', regex=True)  # Remove ponto final
    .str.upper()  # Tudo maiúsculo
    .str.replace(':', '', regex=False) #Remove :
    .str.replace(r'\bREGIÃO:?\b', '', regex=True, flags=re.IGNORECASE)  # Remove REGIÃO ou REGIÃO:
    .str.strip()  # Remove espaços no início e fim
    .str.replace(r'\s+', ' ', regex=True)  # Troca múltiplos espaços por um só
)

Remove o "P" da coluna 'Numero de Projeções'

In [408]:
df['Numero de Projeções'] = df['Numero de Projeções'].str.extract(r'(\d+)').astype(float)
df['quantidade de imagens'] = df['quantidade de imagens'].astype(float)

Preenchendo campos vazios do 'Numero de Projeções' com o numero de imagens extraidas

In [409]:
df['Numero de Projeções'] = df['Numero de Projeções'].fillna(df['quantidade de imagens'])

In [410]:
df.isnull().sum()

ID                          0
Nome do Paciente            0
Nome do Tutor               0
Espécie                     0
Raça                        0
Regiao                      0
Posicionamentos             0
Laudo                       0
Todos Exames                0
Numero de Projeções         0
Data do Exame               0
Imagens                     0
Idade (meses)               0
lista de imagens            0
quantidade de imagens       0
Laudo Alterado              0
Exames Separados            0
Primeira Região             0
Segunda Região            799
Terceira Região          1085
Quarta Região            1090
dtype: int64

Salvar em uma nova planilha o df manipulado

In [411]:
df.to_excel('laudos_piloto002.xlsx', index=False)